In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# FUNCTION FOR CALCULATING ANGLE
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [6]:
# LATERAL RAISE COUNTER
cap = cv2.VideoCapture(0)

# Lateral counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            
            # Calculate angle
            angle1 = calculate_angle(l_wrist, l_shoulder, l_hip)
            angle2 = calculate_angle(r_wrist, r_shoulder, r_hip)
            
            # Visualize angle
            cv2.putText(image, str(angle1), 
                           tuple(np.multiply(l_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(angle2), 
                           tuple(np.multiply(r_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Lateral raise counter logic
            if angle1 < 30 and angle2 < 30:
                stage = "Side"
            if angle1 > 80 and stage =='Side' and angle2 > 80:
                stage="Rest"
                counter +=1
                        
            if angle2 < 30 and angle2 < 80:
                stage = "Side"
            if angle2 > 80 and stage =='Side' and angle1 > 80:
                stage="Rest"
                counter +=1
                
            
                       
        except:
            pass
        
              # Render curl counter
        # Setup status box
#         cv2.rectangle(image, (0,0), (225,73), (0,0,0), -1)
#         img = cv2.imread(image)
        x, y, w, h = 10, 10, 120, 80
        sub_img = image[y:y+h, x:x+w]
        white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 255
        res = cv2.addWeighted(sub_img, 0.5, white_rect, 0.6, 1.0)
        image[y:y+h, x:x+w] = res
        font = cv2.FONT_HERSHEY_TRIPLEX

    
        # Rep data
        cv2.putText(sub_img, 'Direction', (12,18), 
                    font, 0.6, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(sub_img, stage, 
                    (12,60), 
                    font, 0.7, (0,0,0), 1, cv2.LINE_AA)
        
        x, y, w, h = 1150, 10, 120, 80
        sub_img = image[y:y+h, x:x+w]
        white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 255
        res = cv2.addWeighted(sub_img, 0.5, white_rect, 0.6, 1.0)
        image[y:y+h, x:x+w] = res
        
        # Stage data
        cv2.putText(sub_img, 'REPS', (30,18), 
                    font, 0.6, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(sub_img, str(counter), 
                    (30,60), 
                    font, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

17.342842615238286
16.360586720296507
10.481339241317512
4.139122112431372
0.12384589562611266
4.144936524625202
6.0349260137164285
3.7192171159567957
5.284642385906176
4.656628096445159
1.1808715859695162
1.4817578078392846
3.1966974059069404
4.718908611053452
4.378121794231215
4.380623633318756
3.754510448912194
5.231181611700668
9.060625964022217
8.713909668858058
8.859404451541852
7.970844333344845
9.374223287960369
11.917402748058882
12.109355937098345
13.688200902775622
14.3075805722334
8.410010398136453
11.375179393851331
12.151453950742642
12.367265111931665
12.31588533692679
12.963629436973157
16.75022714702083
16.997132227264213
30.436873222606057
32.94640299586528
46.47554582434167
48.70454167390291
69.5364983920739
81.1261614035182
82.09756073872973
90.26404152995154
90.60335102163256
94.95084676345459
96.99886625030811
97.23874490743428
97.42914838075842
97.52436795391807
96.91654726151697
96.91765956974646
94.65798641770539
85.35301722396598
81.87436724090281
68.908643291